In [ ]:
# TODO: Cleanup. remove emojies & don't add volume

import os
import logging
from typing import List, NamedTuple, Optional
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

from eth_defi.provider.multi_provider import create_multi_provider_web3
from eth_defi.gmx.config import GMXConfig
from eth_defi.gmx.api import GMXAPI
from eth_defi.gmx.data import GMXMarketData

logger = logging.getLogger(__name__)


def get_gmx_ohlc_data(config: GMXConfig, token_symbol: str = "ETH", period: str = "1h") -> pd.DataFrame:
    """
    Get OHLC data from GMX API.
    
    :param config: GMX config
    :param token_symbol: Token symbol (ETH, BTC, etc.)
    :param period: Time period (1m, 5m, 15m, 1h, 4h, 1d)
    :return: DataFrame with OHLC data
    """
    
    gmx_api = GMXAPI(config)
    
    # Get raw candlestick data
    raw_data = gmx_api.get_candlesticks(token_symbol, period)
    
    if not raw_data or "candles" not in raw_data:
        return pd.DataFrame()
    
    # Try to extract all available fields
    candles = raw_data["candles"]
    if not candles:
        return pd.DataFrame()
    
    # Determine number of fields in each candle
    num_fields = len(candles[0]) if candles else 0
    
    if num_fields >= 5:
        # At minimum we should have timestamp, open, high, low, close
        columns = ["timestamp", "open", "high", "low", "close"]
        
        df = pd.DataFrame(candles, columns=columns)
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
        
        return df
    
    return pd.DataFrame()


def get_gmx_markets_info(config: GMXConfig) -> dict:
    """Get GMX markets information including TVL and available markets."""
    
    gmx_data = GMXMarketData(config)
    
    # Get available markets
    markets = gmx_data.get_available_markets()
    
    # Get TVL data
    tvl_data = gmx_data.get_pool_tvl()
    
    # Get open interest
    oi_data = gmx_data.get_open_interest()
    
    return {
        "markets": markets,
        "tvl": tvl_data,
        "open_interest": oi_data
    }


def create_ohlc_chart(
    df: pd.DataFrame,
    token_symbol: str,
    market_info: dict = None,
    title_suffix: str = ""
) -> go.Figure:
    """
    Create OHLC chart.
    
    :param df: OHLC DataFrame
    :param token_symbol: Token symbol for chart title
    :param market_info: Additional market information
    :param title_suffix: Additional title information
    :return: Plotly figure
    """
    
    if df.empty:
        return None
    
    # Create figure for OHLC only
    fig = go.Figure()
    
    # Add OHLC candlesticks
    fig.add_trace(
        go.Candlestick(
            x=df["timestamp"],
            open=df["open"],
            high=df["high"],
            low=df["low"],
            close=df["close"],
            name="Price",
            increasing_line_color='green',
            decreasing_line_color='red',
            increasing_fillcolor='rgba(0,255,0,0.3)',
            decreasing_fillcolor='rgba(255,0,0,0.3)'
        )
    )
    
    # Prepare title components
    title_parts = [f"GMX: {token_symbol} OHLC Analysis"]
    
    if title_suffix:
        title_parts.append(title_suffix)
    
    if market_info:
        if "tvl" in market_info and market_info["tvl"]:
            # Find matching market TVL
            for market_symbol, tvl_data in market_info["tvl"].items():
                if token_symbol.upper() in market_symbol.upper():
                    tvl_value = tvl_data['total_tvl']
                    
                    # Fix obviously wrong TVL values (likely decimal issues)
                    if tvl_value > 1e15:  # If TVL > $1 quadrillion, it's wrong
                        # Try dividing by 1e18 (wei conversion)
                        tvl_value = tvl_value / 1e18
                        if tvl_value > 1e12:  # Still too big, try 1e30
                            tvl_value = tvl_data['total_tvl'] / 1e30
                    
                    # Only show if reasonable (between $1K and $100B)
                    if 1000 <= tvl_value <= 1e11:
                        title_parts.append(f"TVL: ${tvl_value:,.0f}")
                    else:
                        title_parts.append(f"TVL: Debug({tvl_data['total_tvl']:.2e})")
                    break
    
    # Update layout
    fig.update_layout(
        title=" | ".join(title_parts),
        height=500,
        showlegend=False,
        xaxis_rangeslider_visible=False
    )
    
    # Update axes
    fig.update_yaxes(title_text="Price (USD)")
    fig.update_xaxes(title_text="Time")
    
    # Format price axis
    fig.update_yaxes(tickformat=".2f")
    
    return fig


def analyze_gmx_token_ohlc(
    config: GMXConfig,
    token_symbol: str = "ETH",
    period: str = "1h"
) -> dict:
    """
    Complete OHLC analysis for a GMX token.
    
    :param config: GMX configuration
    :param token_symbol: Token to analyze
    :param period: Time period for candles
    :return: Analysis results
    """
    
    print(f"Analyzing {token_symbol} OHLC data...")
    
    # 1. Get OHLC data from GMX API
    ohlc_df = get_gmx_ohlc_data(config, token_symbol, period)
    
    if ohlc_df.empty:
        print(f"No OHLC data available for {token_symbol}")
        return {}
    
    print(f"Retrieved {len(ohlc_df)} {period} candles")
    
    # 2. Get market information
    market_info = get_gmx_markets_info(config)
    
    # 3. Create OHLC chart
    fig = create_ohlc_chart(ohlc_df, token_symbol, market_info)
    
    # 4. Display chart
    if fig:
        fig.show()
    
    # 5. Print summary statistics
    print(f"\n{token_symbol} OHLC Summary:")
    print(f"Data period: {ohlc_df['timestamp'].min()} to {ohlc_df['timestamp'].max()}")
    print(f"Price range: ${ohlc_df['low'].min():.2f} - ${ohlc_df['high'].max():.2f}")
    print(f"Latest price: ${ohlc_df['close'].iloc[-1]:.2f}")
    
    # Price statistics
    price_change = ohlc_df['close'].iloc[-1] - ohlc_df['open'].iloc[0]
    price_change_pct = (price_change / ohlc_df['open'].iloc[0]) * 100
    print(f"Price change: ${price_change:+.2f} ({price_change_pct:+.2f}%)")
    
    return {
        "ohlc_data": ohlc_df,
        "market_info": market_info,
        "chart": fig,
        "stats": {
            "price_range": (ohlc_df['low'].min(), ohlc_df['high'].max()),
            "price_change": price_change,
            "price_change_pct": price_change_pct
        }
    }


def compare_multiple_tokens(config: GMXConfig, tokens: List[str] = ["ETH", "BTC"], period: str = "1h"):
    """Compare OHLC data for multiple tokens."""
    
    results = {}
    
    for token in tokens:
        print(f"\n{'='*50}")
        results[token] = analyze_gmx_token_ohlc(config, token, period)
    
    # Create comparison summary
    print(f"\nComparison Summary:")
    print("-" * 60)
    
    for token, result in results.items():
        if result and "stats" in result:
            stats = result["stats"]
            print(f"{token:>6}: ${stats['price_change']:+8.2f} ({stats['price_change_pct']:+6.2f}%)")
    
    return results


def example_gmx_ohlc_analysis():
    """Example OHLC analysis."""
    
    # Setup connection
    json_rpc_url = os.environ.get("ARBITRUM_CHAIN_JSON_RPC")
    if not json_rpc_url:
        json_rpc_url = input("Please enter your Arbitrum JSON-RPC URL: ")
    
    web3 = create_multi_provider_web3(json_rpc_url)
    config = GMXConfig(web3)
    
    print(f"Connected to chain {web3.eth.chain_id}")
    
    # Quick freshness check
    print(f"\nData Freshness Check:")
    current_block = web3.eth.block_number
    print(f"Current block: {current_block:,}")
    
    # Single token analysis
    results = analyze_gmx_token_ohlc(config, "ETH", "1h")
    
    # Multiple token comparison
    print(f"\n{'='*60}")
    print("MULTI-TOKEN COMPARISON")
    compare_multiple_tokens(config, ["LINK", "BTC", "ARB", "DOGE"], "1h")
    
    return results


if __name__ == "__main__":
    example_gmx_ohlc_analysis()